<a href="https://colab.research.google.com/github/prince24-web/Google-colabs/blob/main/Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
from langchain.document_loaders import PyPDFLoader
drive.mount('/content/drive')

pdf_path = '/content/drive/MyDrive/zero_one.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(len(docs))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
166


In [38]:
%pip install pypdf

In [39]:
%pip install -U langchain-community

In [40]:
print(f"{docs[7].page_content[:200]}\n")
print(docs[7].metadata)

radically in the next decade, then the future is nearly at hand. No one can
predict the future exactly, but we know two things: it’s going to be diﬀerent,
and it must be rooted in today’s world. Most 

{'producer': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creator': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creationdate': '2020-03-06T12:48:07+00:00', 'author': 'Peter Thiel & Blake Masters', 'keywords': 'Non-Fiction', 'moddate': '2020-03-06T17:48:07+05:00', 'title': 'Zero to One: Notes on Startups, or How to Build the Future', 'source': '/content/drive/MyDrive/zero_one.pdf', 'total_pages': 166, 'page': 7, 'page_label': '8'}


In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

369

In [42]:
print(f"{all_splits[7].page_content[-200:]}\n")
print(all_splits[7].metadata)
print(f"{all_splits[8].page_content[:200]}\n")
print(all_splits[8].metadata)

nd the campus, and in Zero to One I have worked
with him to revise the notes for a wider audience. There’s no reason why the
future should happen only at Stanford, or in college, or in Silicon Valley.

{'producer': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creator': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creationdate': '2020-03-06T12:48:07+00:00', 'author': 'Peter Thiel & Blake Masters', 'keywords': 'Non-Fiction', 'moddate': '2020-03-06T17:48:07+05:00', 'title': 'Zero to One: Notes on Startups, or How to Build the Future', 'source': '/content/drive/MyDrive/zero_one.pdf', 'total_pages': 166, 'page': 5, 'page_label': '6', 'start_index': 788}
W
1
THE CHALLENGE OF THE FUTURE
HENEVER I INTERVIEW someone for a job, I like to ask this question:
“What important truth do very few people agree with you on?”
This question sounds easy because it’s 

{'producer': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creator': 'calibre (2.3.0) [http://calibre-ebook.com]', 'creationdate': '2020

In [43]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [44]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vector of length {len(vector_1)}")
print(vector_1[:5])

Generated vector of length 768
[0.03999001532793045, 0.0742223858833313, -0.036845918744802475, -0.019633622840046883, -0.004903456196188927]


In [45]:
%pip install -qU langchain-core

In [46]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
id = vector_store.add_documents(documents=all_splits)

In [ ]:
result = vector_store.similarity_search("DOT-COM BOOM")
print(result[0])

In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("what is vertical progress")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
embedding = embeddings.embed_query("what is vertical progress")
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

In [ ]:
from re import search
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
  return vector_store.as_retriever(
      search_string=query,
      search_type="similarity",
      search_kwargs={"k": 1},
  )
queries = [
    "What caused the dot-com crash?",
    "When did the internet bubble burst?",
    "What was the outcome of the dot-com boom?",
]

results = retriever.batch(queries)

retriever.batch(queries)

for i, docs in enumerate(results):
    print(f"\n🔍 Query: {queries[i]}")
    for doc in docs:
        print("📄", doc.page_content[:200], "...\n")
